In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [ ]:
url= "https://forbesafrique.com/"
html = requests.get(url)

soup = BeautifulSoup(html.text, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html lang="fr-FR">
 <head>
  <meta charset="utf-8"/>
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
   <style>
    img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }
   </style>
   <meta content="width=device-width, initial-scale=1" name="viewport"/>
   <!-- This site is optimized with the Yoast SEO plugin v26.3 - https://yoast.com/wordpress/plugins/seo/ -->
   <title>
    Forbes Afrique, les acteurs inspirants de l'économie africaine - Forbes Afrique
   </title>
   <meta content="Le média de ceux qui font l'Afrique d'aujourd'hui et construisent celle de demain. Enquêtes, portraits, classements : décryptage de l’économie du continent." name="description">
    <link href="https://forbesafrique.com/" rel="canonical"/>
    <meta content="fr_FR" property="og:locale"/>
    <meta content="website" property="og:type"/>
    <meta content="Forbes Afrique, les acteurs inspirants

In [10]:
# --- CONFIG ---

base_url = 'https://forbesafrique.com'  
max_articles = 50   

# listes de sortie
sources, titres, dates, liens, textes = [], [], [], [], []

# --- CHARGER LA PAGE ---

r = requests.get(base_url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=15)
r.encoding = 'utf-8'
html = r.text

soup = BeautifulSoup(html, 'html.parser')

# --- TROUVER LES LIENS CANDIDATS (STRATÉGIE SIMPLE) ---
# On cherche les <a> qui ont href contenant le site et qui ont un texte significatif
candidates = []
for a in soup.find_all('a', href=True):
    href = a['href'].strip()
    text = (a.get_text(strip=True) or '')
    if not href:
        continue
    # heuristiques : lien d'article typique sur Forbes : /yyyy/... ou /slug-article/ ou contient 'forbesafrique.com'
    if (href.startswith('/') or 'forbesafrique.com' in href) and len(text) > 8:
        # normaliser les urls relatives
        full = href if href.startswith('http') else urljoin(base_url, href)
        candidates.append({'url': full, 'anchor_text': text})

# dédupliquer tout en conservant ordre
seen = set()
unique_candidates = []
for c in candidates:
    u = c['url']
    if u not in seen:
        unique_candidates.append(c)
        seen.add(u)

print(f"Liens candidats trouvés : {len(unique_candidates)} (après dédoublonnage)")

# limiter au nombre max_articles
unique_candidates = unique_candidates[: max_articles * 3]  # rechercher un peu plus au cas où certains échouent

# --- POUR CHAQUE CANDIDAT : ouvrir la page et tenter d'extraire titre, date, texte ---
for i, cand in enumerate(unique_candidates):
    if len(titres) >= max_articles:
        break

    url = cand['url']
    print(f"\n[{len(titres)+1}] Tentative -> {url}")

    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=12)
        if r.status_code != 200:
            print("statut:", r.status_code, "- sauter")
            continue
        r.encoding = 'utf-8'
        page = r.text
        psoup = BeautifulSoup(page, 'html.parser')

        # TITRE : tenter plusieurs sélecteurs (Forbes utilise souvent h1.entry-title ou .elementor-heading-title)
        titre = None
        sel_titre = psoup.select_one('h1.entry-title') or psoup.select_one('h1') or psoup.select_one('.elementor-heading-title') or psoup.select_one('title')
        if sel_titre:
            titre = sel_titre.get_text(strip=True)
        else:
            titre = cand['anchor_text'] or 'Titre non trouvé'

        # DATE : essayer meta/article date, ou sélecteurs usuels
        date = None
        # meta property
        meta_dt = psoup.find('meta', {'property': 'article:published_time'}) or psoup.find('meta', {'name':'pubdate'}) or psoup.find('meta', {'name':'date'})
        if meta_dt and meta_dt.has_attr('content'):
            date = meta_dt['content'].strip()
        else:
            # chercher un span ou div contenant une date "DD Month YYYY" ou pattern YYYY
            possible = psoup.find_all(text=True)
            import re
            for t in possible:
                txt = t.strip()
                if not txt:
                    continue
                # ex: "5 novembre 2025" ou "2025-11-05"
                if re.search(r'\b\d{1,2}\s+[A-Za-zéûîôäàè]+\s+\d{4}\b', txt) or re.search(r'\b\d{4}-\d{2}-\d{2}\b', txt):
                    date = txt
                    break
            if not date:
                # fallback : chercher .post-date, .meta, .entry-meta
                sel = psoup.select_one('.post-date') or psoup.select_one('.entry-meta time') or psoup.select_one('.post-meta') or psoup.select_one('.meta__date')
                if sel:
                    date = sel.get_text(strip=True)
        if not date:
            date = 'N/A'

        # TEXTE : rassembler les <p> à l'intérieur d'un conteneur article
        texte = ''
        container = psoup.select_one('article') or psoup.select_one('.entry-content') or psoup.select_one('.post-content') or psoup.select_one('.content')
        if container:
            ps = container.find_all('p')
            if ps:
                texte = "\n\n".join([p.get_text(strip=True) for p in ps if p.get_text(strip=True)])
            else:
                texte = container.get_text(separator='\n', strip=True)
        else:
            # fallback : rassembler premiers <p> du document (danger: nav ou sidebar)
            ps = psoup.find_all('p')
            if ps:
                # prendre les premiers paragraphes significatifs
                collected = []
                for p in ps[:30]:
                    t = p.get_text(strip=True)
                    if t and len(t) > 20:
                        collected.append(t)
                texte = "\n\n".join(collected)
            else:
                texte = 'Texte non disponible'

        # réduire la longueur pour l'export si tu veux (ici on conserve raisonnablement)
        if len(texte) > 20000:
            texte = texte[:20000] + '...'

        # heuristique simple pour décider si c'est bien un article (titre présent + texte suffisant)
        if titre and titre != 'Titre non trouvé' and len(texte) > 150:
            sources.append('Forbes Afrique')
            titres.append(titre)
            dates.append(date)
            liens.append(url)
            textes.append(texte)
            print(f"Article capturé : '{titre[:70]}' (texte len={len(texte)})")
        else:
            print("Non considéré comme article (titre ou texte insuffisant), ignorer")

        # courte pause pour ne pas surcharger le site
        time.sleep(0.8)

    except Exception as e:
        print("  rreur pendant la récupération :", str(e))
        continue

# --- RÉSULTAT ---
df = pd.DataFrame({
    'source': sources,
    'titre': titres,
    'date': dates,
    'lien': liens,
    'texte': textes
})

print("\n--- Résumé ---")
print(f"Articles récupérés : {len(df)}")
if len(df) > 0:
    print(df[['titre','date','lien']].head(10))

# Sauvegarde
out_csv = '../outputs/forbes_articles.csv'
#out_xlsx = 'forbes_articles_extracted.xlsx'
df.to_csv(out_csv, index=False, encoding='utf-8-sig')
print("CSV écrit :", out_csv)

# df.to_excel(out_xlsx, index=False, engine='openpyxl')
# print(" Excel écrit :", out_xlsx)

Liens candidats trouvés : 51 (après dédoublonnage)

[1] Tentative -> https://forbesafrique.com/cover-stories/


C:\Users\ulric\AppData\Local\Temp\ipykernel_15752\1737439658.py:78: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  possible = psoup.find_all(text=True)


Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=629)

[2] Tentative -> https://forbesafrique.com/milliardaires/
Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=485)

[3] Tentative -> https://forbesafrique.com/portrait-et-interview/
Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=277)

[4] Tentative -> https://forbesafrique.com/classement/
Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=162)

[5] Tentative -> https://forbesafrique.com/technologie/
Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=606)

[6] Tentative -> https://forbesafrique.com/blockchain/
Article capturé : 'Le média de ceux qui construisent l'Afrique d'aujourd'hui et de demain' (texte len=364)

[7] Tentative -> https://forbesafrique.com/management/
Article